# PyPSA Incremental Model Builder - Interactive Tutorial

This notebook teaches you how to build PyPSA energy system models incrementally, starting from an empty network and progressively adding components while understanding how each piece affects the model.

## Learning Objectives

By the end of this tutorial, you will understand:
1. **Component Types**: Generators vs Links vs Lines vs Stores
2. **Technology Mapping**: Which technologies use which component types
3. **Balance Validation**: How supply and demand balance works
4. **Optimization**: What makes a model feasible or infeasible
5. **Incremental Building**: How to test and understand each step

## Prerequisites

- Basic Python knowledge
- Understanding of energy systems (generation, demand, storage)
- Familiarity with PyPSA concepts (helpful but not required)

---

## 🔧 Setup: Import Required Modules

First, we need to import the necessary libraries and set up logging so we can see what's happening.

In [1]:
# Standard library imports
import sys
import logging
from pathlib import Path

# Configure logging to show INFO level messages
# This lets us see what the incremental builder is doing at each step
logging.basicConfig(
    level=logging.INFO,  # Show informational messages
    format='%(levelname)s: %(message)s'  # Simple format: "INFO: message"
)

# Import the incremental builder
# This is our main tool for building PyPSA models step-by-step
from incremental_builder import IncrementalBuilder

print("✓ Imports successful!")
print("Ready to build PyPSA models incrementally.")

INFO: NumExpr defaulting to 16 threads.


✓ Imports successful!
Ready to build PyPSA models incrementally.


---

## 📚 Understanding PyPSA Component Types (CRITICAL!)

Before we start building, you **must** understand that different technologies use different PyPSA component types:

### 1. **Generators**
- **Supply**: Renewables (wind, solar), nuclear, supply sources
- **Demand**: Final use (modeled as negative generators with sign < 0)
- **Examples**: `'wind onshore'`, `'solar PV ground'`, `'electricity final use'`
- **Key Property**: Inject or withdraw power at a bus

### 2. **Links**
- **Thermal Plants**: Coal, gas, biomass power plants (fuel → electricity)
- **Conversion**: Heat pumps, electrolysis, CHPs
- **Examples**: `'hard coal power old'`, `'natural gas power CCGT'`, `'heat pump large AW'`
- **Key Property**: Convert one energy carrier to another with efficiency

### 3. **Lines**
- **Transmission**: High-voltage transmission with impedance
- **Examples**: `'transmission line AC'`

### 4. **Stores**
- **Storage**: Batteries, pumped hydro, hydrogen storage
- **Examples**: `'battery large storage'`, `'hydro PSH storage'`

### ⚠️ Common Mistake
**Coal/gas plants are Links, NOT Generators!** They convert fuel (input carrier) to electricity (output carrier).

### 📝 Technology Names Use SPACES
- ✓ Correct: `'hard coal power old'`, `'wind onshore'`
- ✗ Wrong: `'hard_coal_power_old'`, `'wind_onshore'`

---

## Step 1: Create the Incremental Builder

The `IncrementalBuilder` class is our main interface. Let's create one and understand each parameter.

In [2]:
# Create an incremental builder instance
builder = IncrementalBuilder(
    year=2025,           # Model year: 2025, 2030, 2035, or 2040
                         # Different years have different technology availability
                         # and policy constraints
    
    timeseries='mini',   # Time resolution:
                         # 'mini' = 1 week (168 hours) - fastest, for learning
                         # 'medium' = ~1 month - more realistic
                         # 'full' = full year - production runs
    
    copperplate=True     # Spatial resolution:
                         # True = Single PL area (simpler, faster)
                         # False = 16 voivodeships (more detailed)
)

print("\n✓ Builder created!")
print(f"  Year: {builder.year}")
print(f"  Time resolution: {builder.timeseries}")
print(f"  Spatial resolution: {'Copperplate (single PL)' if builder.copperplate else 'Voivodeships (16 regions)'}")

2026-02-08 19:14:18 [INFO] Initialized IncrementalBuilder: year=2025, timeseries=mini, copperplate=True



✓ Builder created!
  Year: 2025
  Time resolution: mini
  Spatial resolution: Copperplate (single PL)


### 🔍 What Just Happened?

The builder:
1. **Loaded parameters**: Created a complete parameter dictionary with all required settings
2. **Set up data paths**: Configured which input files to use based on year and resolution
3. **Initialized storage**: Prepared containers for network components

**Note**: No data is loaded yet - that happens in the next step when we build the base model.

---

## Step 2: Build the Base Model Structure

This creates an **empty** network with structure but **no components** yet. Think of it as building the framework before adding the pieces.

In [3]:
# Build the base model structure
# This will:
# 1. Load input data from CSV files (6 files total)
# 2. Create custom PyPSA network with extended attributes
# 3. Add time dimension (snapshots)
# 4. Add energy carriers (electricity, heat, hydrogen, etc.)
# 5. Add buses (network nodes where energy is injected/withdrawn)
# 6. Process capacity data (merge technologies with costs/constraints)
#
# What it does NOT do yet:
# - Add generators, links, lines, or stores (that's next!)

builder.build_base_model()

2026-02-08 19:14:18 [INFO] ============================================================
2026-02-08 19:14:18 [INFO] STAGE 0: Building base model structure
2026-02-08 19:14:18 [INFO] ============================================================
2026-02-08 19:14:18 [INFO] Loading and preprocessing inputs...
/home/magda/projects/pyPSA-PL/pypsa-pl/src/pypsa_pl/build_network.py:64: FutureWarning:

errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead

2026-02-08 19:14:18 [INFO] Loaded 8 input datasets
2026-02-08 19:14:18 [INFO] Creating custom network...
2026-02-08 19:14:18 [INFO] Adding snapshots...
2026-02-08 19:14:18 [INFO] Adding carriers...
2026-02-08 19:14:18 [INFO] Adding buses and areas...
2026-02-08 19:14:18 [INFO] Processing capacity data...
2026-02-08 19:14:18 [INFO] Base model structure created successfully!



NETWORK SUMMARY
Network components:
  Areas:      6
  Buses:      92
  Carriers:   106
  Snapshots:  672
  Generators: 0
  Links:      0
  Lines:      0
  Stores:     0
  Global constraints: 0


### 🔍 Understanding the Output

The log messages show:
1. **Loading inputs**: Reading CSV files with technology definitions, costs, capacities, etc.
2. **Creating network**: Building PyPSA Network object with custom attributes (like 'area', 'technology')
3. **Adding snapshots**: Creating time index (168 hours for 'mini')
4. **Adding carriers**: Energy types that flow through the network
5. **Adding buses**: Nodes where components connect
6. **Processing capacity data**: Preparing component data for later addition

Let's inspect what we have:

In [4]:
# Inspect the base model structure
builder.inspect('summary')


NETWORK SUMMARY
Network components:
  Areas:      6
  Buses:      92
  Carriers:   106
  Snapshots:  672
  Generators: 0
  Links:      0
  Lines:      0
  Stores:     0
  Global constraints: 0



### 📊 Reading the Summary

- **Buses**: Network nodes (e.g., "PL electricity in", "PL electricity out EHV")
- **Carriers**: Energy types (e.g., "electricity in", "electricity out", "heat centralised")
- **Snapshots**: Time periods (168 hours for mini)
- **Generators/Links/Lines/Stores**: All 0! We haven't added components yet.

This is like having a road network (buses and connections) but no vehicles (generators/links) yet.

---

## Step 3: Add First Components - Coal Generation

Let's add coal power plants. **Remember**: Thermal plants are **Links**, not Generators!

### Why are coal plants Links?
Coal plants convert **hard coal** (input carrier) → **electricity** (output carrier). This conversion is modeled as a Link with efficiency.

In [5]:
# Add coal fuel supply + power plants
# 
# CRITICAL: Thermal Links need FUEL to operate!
# Step 1: Add unlimited fuel supply (Generator)
# Step 2: Add power plants (Link) that convert fuel → electricity
#
# This creates the complete chain:
# Fuel supply (Generator) → Fuel bus → Power plant (Link) → Electricity bus

# Step 1: Add coal fuel supply
print("Step 1: Adding coal fuel supply...")
summary = builder.add_components(
    'Generator',
    {
        'technology': ['hard coal supply'],  # Provides coal to 'PL hard coal' bus
        'area': ['PL']
    }
)
print(f"  ✓ Added {summary['added']} fuel supply components")

# Step 2: Add coal power plants
print("\nStep 2: Adding coal power plants...")
summary = builder.add_components(
    'Link',  # Component type: Link (thermal conversion)
    {
        'technology': ['hard coal power old', 'hard coal power SC'],
        'area': ['PL']  # Only Poland - exclude foreign countries
    }
)

print(f"  ✓ Added {summary['added']} coal power plant components")
print(f"\nTotal network components:")
print(f"  Generators: {summary['generators']}")
print(f"  Links: {summary['links']}")

2026-02-08 19:14:18 [INFO] Defining time-dependent attributes...
2026-02-08 19:14:18 [INFO] Adding 1 components to network...
2026-02-08 19:14:18 [INFO] Added 1 components. Network now has: 1 generators, 0 links, 0 lines, 0 stores
2026-02-08 19:14:18 [INFO] Defining time-dependent attributes...
2026-02-08 19:14:18 [INFO] Adding 2 components to network...
2026-02-08 19:14:18 [INFO] Added 2 components. Network now has: 1 generators, 2 links, 0 lines, 0 stores


Step 1: Adding coal fuel supply...
  ✓ Added 1 fuel supply components

Step 2: Adding coal power plants...
  ✓ Added 2 coal power plant components

Total network components:
  Generators: 1
  Links: 2


Data Flow: [analiza jak builder.add_components() method dodaje dane z plikow csv w oparciu o filtr ]
Step 1: Each CSV → One DataFrame

df = read_input("installed_capacity", "historical_totals")
Returns: DataFrame with columns like [area, technology, p_nom, ...]

# Step 2: DataFrames → Dictionary


inputs = {
    "technology_carrier_definitions": df1,  # DataFrame
    "technology_cost_data": df2,            # DataFrame
    "installed_capacity": df3,              # DataFrame
    "annual_energy_flows": df4,             # DataFrame
    "capacity_utilisation": df5,            # DataFrame
    "capacity_addition_potentials": df6,    # DataFrame
}


# Step 3: Dictionary → builder.inputs

So: builder.inputs is a dictionary of separate dataframes!

##  Step 4: Merge & Process → builder.df_cap_full

This merges all the separate dataframes into ONE big dataframe with all capacity data!

## Summary - Where Data Lives in builder: 

builder.inputs = {
    "technology_carrier_definitions": DataFrame,  # ← Separate dataframes
    "technology_cost_data": DataFrame,            #    from each CSV
    "installed_capacity": DataFrame,
    "annual_energy_flows": DataFrame,
    "capacity_utilisation": DataFrame,
    "capacity_addition_potentials": DataFrame,
}

builder.df_cap_full = DataFrame  # ← ALL capacity data merged
                                  #    (created from builder.inputs)
                                  #    This is what you filter from!

builder.df_cap = DataFrame       # ← Currently added components
                                  #    (starts empty, grows as you add)

builder.network                   # ← The actual PyPSA Network object

# To See Them - run below

In [6]:
# See what's in builder
print("builder.inputs keys:", builder.inputs.keys())
print(f"\nbuilder.df_cap_full shape: {builder.df_cap_full.shape}")
print(f"builder.df_cap_full columns: {builder.df_cap_full.columns.tolist()}")
print(f"\nFirst 5 rows of capacity data:")
print(builder.df_cap_full.head())


builder.inputs keys: dict_keys(['technology_carrier_definitions', 'technology_cost_data', 'installed_capacity', 'annual_energy_flows', 'capacity_utilisation', 'capacity_addition_potentials', 'co2_cost', 'final_use'])

builder.df_cap_full shape: (242, 58)
builder.df_cap_full columns: ['name', 'area', 'area_from', 'technology', 'qualifier', 'build_year', 'retire_year', 'cumulative', 'nom', 'length', 'carrier', 'aggregation', 'component', 'bus', 'sign', 'p_nom', 'bus_input', 'bus_output', 'bus_output2', 's_nom', 'e_nom', 'technology_year', 'availability_correction_factor', 'co2_emissions', 'efficiency', 'efficiency2', 'fixed_cost', 'investment_cost', 'lifetime', 'line_type', 'p_max_pu_annual', 'parent_ratio', 'standing_loss', 'variable_cost', 'co2_cost', 'marginal_cost', 'annual_investment_cost', 'capital_cost', 'p_nom_extendable', 'p_nom_min', 'p_nom_max', 's_nom_extendable', 's_nom_min', 's_nom_max', 'e_nom_extendable', 'e_nom_min', 'e_nom_max', 'p_set', 'p_set_annual', 'p_max_pu', 'p_m

**TL;DR:** 6 CSVs → 6 dataframes in builder.inputs dict
Merged into 1 big dataframe: builder.df_cap_full
Your cell [5] / Step 3 filters from builder.df_cap_full


In [7]:
# Display coal power plant components
coal_links = builder.network.links[
    builder.network.links['technology'].str.contains('hard coal power')
]

print("Coal Power Plants Added:")
print("="*80)
print(coal_links[['bus0', 'bus1', 'p_nom', 'efficiency', 'technology', 'area']])
print("\n")
print(f"Total capacity: {coal_links['p_nom'].sum():.2f} MW")


Coal Power Plants Added:
                                                         bus0          bus1  \
Link                                                                          
PL hard coal power SC public EHV 2025   PL electricity in EHV  PL hard coal   
PL hard coal power old public EHV 2025  PL electricity in EHV  PL hard coal   

                                          p_nom  efficiency  \
Link                                                          
PL hard coal power SC public EHV 2025    4050.0    2.325581   
PL hard coal power old public EHV 2025  10520.0    3.030303   

                                                 technology area  
Link                                                              
PL hard coal power SC public EHV 2025    hard coal power SC   PL  
PL hard coal power old public EHV 2025  hard coal power old   PL  


Total capacity: 14570.00 MW


### 🔍 What Happened?

1. **Filtered capacity data**: Found all coal power plants in the input data
2. **Added to network**: Created Link components with:
   - `bus0`: Input bus (hard coal)
   - `bus1`: Output bus (electricity in)
   - `efficiency`: Conversion efficiency
   - `p_nom`: Nominal power capacity (MW)
3. **Updated network**: Network now has Links but still no demand

Let's check the balance:

In [8]:
# Check supply/demand balance
# This shows:
# - Total supply capacity (from Generators + Links)
# - Total demand capacity (from negative Generators)
# - Balance ratio (supply/demand)

builder.inspect('balance')


SUPPLY/DEMAND BALANCE
Total supply capacity:    1014569.00 MW
  From Generators:         999999.00 MW
  From Links:               14570.00 MW (thermal plants)
Total demand capacity:          0.00 MW

⚠️  WARNING: No demand in network

Generator Supply by Carrier
---------------------------
  hard coal supply                        :  999999.00 MW

Link Supply by Technology (Thermal Plants)
------------------------------------------
  hard coal power SC                      :    4050.00 MW
  hard coal power old                     :   10520.00 MW


In [9]:
# Check if any Links exist
print(f"Number of Links: {len(builder.network.links)}")

# Show the coal plants
if len(builder.network.links) > 0:
    print("\nCoal plants in network:")
    print(builder.network.links[['bus0', 'bus1', 'p_nom', 'efficiency', 'technology']])
else:
    print("No Links found! Coal plants were NOT added.")


Number of Links: 2

Coal plants in network:
                                                         bus0          bus1  \
Link                                                                          
PL hard coal power SC public EHV 2025   PL electricity in EHV  PL hard coal   
PL hard coal power old public EHV 2025  PL electricity in EHV  PL hard coal   

                                          p_nom  efficiency  \
Link                                                          
PL hard coal power SC public EHV 2025    4050.0    2.325581   
PL hard coal power old public EHV 2025  10520.0    3.030303   

                                                 technology  
Link                                                         
PL hard coal power SC public EHV 2025    hard coal power SC  
PL hard coal power old public EHV 2025  hard coal power old  


In [10]:
# Check what components have been added to df_cap
print(f"\nComponents in builder.df_cap: {len(builder.df_cap)}")
if len(builder.df_cap) > 0:
    print(builder.df_cap[['name', 'component', 'area', 'technology', 'p_nom']])



Components in builder.df_cap: 3
                                     name  component area  \
0                PL hard coal supply 2025  Generator   PL   
1   PL hard coal power SC public EHV 2025       Link   PL   
2  PL hard coal power old public EHV 2025       Link   PL   

            technology     p_nom  
0     hard coal supply  999999.0  
1   hard coal power SC    4050.0  
2  hard coal power old   10520.0  


In [11]:
# Run inspect AFTER adding components
builder.inspect('balance')



SUPPLY/DEMAND BALANCE
Total supply capacity:    1014569.00 MW
  From Generators:         999999.00 MW
  From Links:               14570.00 MW (thermal plants)
Total demand capacity:          0.00 MW

⚠️  WARNING: No demand in network

Generator Supply by Carrier
---------------------------
  hard coal supply                        :  999999.00 MW

Link Supply by Technology (Thermal Plants)
------------------------------------------
  hard coal power SC                      :    4050.00 MW
  hard coal power old                     :   10520.00 MW


### 📊 Understanding the Balance Report

You should see:
- **Supply from Generators**: 0 MW (we haven't added renewable generators yet)
- **Supply from Links**: ~X MW (from coal plants we just added)
- **Demand**: 0 MW (we haven't added demand yet!)
- **Warning**: No demand in network

**Key Insight**: The balance report now correctly counts **both** Generators and Links as supply sources. This was one of the critical fixes!

---

## Step 4: Add Electricity Demand

Now let's add demand. **Demand is modeled as negative Generators** (sign < 0).

In [12]:
# See all available carriers for Generators
generators_in_data = builder.df_cap_full[builder.df_cap_full['component'] == 'Generator']
print("Available Generator carriers:")
print(generators_in_data['carrier'].unique())


Available Generator carriers:
['DSR reduction' 'biomass wood supply' 'electricity final use'
 'hard coal supply' 'hydro ROR' 'lignite supply' 'natural gas supply'
 'nuclear power' 'solar PV ground' 'wind onshore' 'hydrogen final use'
 'wind offshore' 'ICE vehicle fuel supply' 'biogas substrate supply'
 'biomass agriculture supply' 'biomass wood final use'
 'building retrofits' 'electricity HMV final use'
 'electricity LV final use' 'hard coal final use'
 'light vehicle mobility final use' 'lulucf final use' 'lulucf supply'
 'natural gas final use' 'other RES heat' 'other fuel final use'
 'other fuel supply' 'other heating final use'
 'process emissions final use' 'process emissions supply' 'solar PV roof'
 'space heating final use' 'water heating final use']


In [13]:
# Search for demand-related carriers
demand_carriers = builder.df_cap_full['carrier'].unique()
print("All carriers containing 'final use':")
for c in demand_carriers:
    if 'final use' in c:
        print(f"  - {c}")


All carriers containing 'final use':
  - electricity final use
  - hydrogen final use
  - biomass wood final use
  - electricity HMV final use
  - electricity LV final use
  - hard coal final use
  - light vehicle mobility final use
  - lulucf final use
  - natural gas final use
  - other fuel final use
  - other heating final use
  - process emissions final use
  - space heating final use
  - water heating final use


In [14]:
# Add electricity demand
#
# Component type: Generator (with negative sign!)
# Why: PyPSA models demand as generators with sign < 0
#      This represents energy withdrawal from the network
#
# Carrier: 'electricity final use' > changed to HMV and LV existing in csv data ! 
# This represents end-user electricity consumption:
# - Households
# - Industry
# - Services
# (but NOT electric vehicles, heat pumps - those are separate)

# Add electricity demand (both voltage levels)
summary = builder.add_components(
    'Generator',
    {
        'carrier': [
            'electricity HMV final use',  # High/Medium voltage
            'electricity LV final use'    # Low voltage
        ],
        'area': ['PL']
    }
)

print(f"\n✓ Added {summary['added']} demand components")
print(f"  Total Generators in network: {summary['generators']}")


2026-02-08 19:14:18 [INFO] Defining time-dependent attributes...
2026-02-08 19:14:18 [INFO] Adding 2 components to network...
2026-02-08 19:14:19 [INFO] Added 2 components. Network now has: 3 generators, 2 links, 0 lines, 0 stores



✓ Added 2 demand components
  Total Generators in network: 3


In [15]:
# See what's in builder
print("builder.inputs keys:", builder.inputs.keys())
print(f"\nbuilder.df_cap_full shape: {builder.df_cap_full.shape}")
print(f"builder.df_cap_full columns: {builder.df_cap_full.columns.tolist()}")
print(f"\nFirst 5 rows of capacity data:")
print(builder.df_cap_full.head())


builder.inputs keys: dict_keys(['technology_carrier_definitions', 'technology_cost_data', 'installed_capacity', 'annual_energy_flows', 'capacity_utilisation', 'capacity_addition_potentials', 'co2_cost', 'final_use'])

builder.df_cap_full shape: (242, 58)
builder.df_cap_full columns: ['name', 'area', 'area_from', 'technology', 'qualifier', 'build_year', 'retire_year', 'cumulative', 'nom', 'length', 'carrier', 'aggregation', 'component', 'bus', 'sign', 'p_nom', 'bus_input', 'bus_output', 'bus_output2', 's_nom', 'e_nom', 'technology_year', 'availability_correction_factor', 'co2_emissions', 'efficiency', 'efficiency2', 'fixed_cost', 'investment_cost', 'lifetime', 'line_type', 'p_max_pu_annual', 'parent_ratio', 'standing_loss', 'variable_cost', 'co2_cost', 'marginal_cost', 'annual_investment_cost', 'capital_cost', 'p_nom_extendable', 'p_nom_min', 'p_nom_max', 's_nom_extendable', 's_nom_min', 's_nom_max', 'e_nom_extendable', 'e_nom_min', 'e_nom_max', 'p_set', 'p_set_annual', 'p_max_pu', 'p_m

In [16]:
# Check for PL electricity demand - this is trouble shooting - previously only 'electricity final use' was being looked up - zamiast hmv / lv
demand = builder.df_cap_full[
    (builder.df_cap_full['component'] == 'Generator') &
    (builder.df_cap_full['carrier'] == 'electricity final use') &
    (builder.df_cap_full['area'] == 'PL')
]

print(f"Matching rows: {len(demand)}")
if len(demand) > 0:
    print("\nDemand components found:")
    print(demand[['name', 'area', 'carrier', 'component', 'p_nom']])
else:
    print("\nNo matches! Checking each filter separately...")
    
    # Check component type
    gens = builder.df_cap_full[builder.df_cap_full['component'] == 'Generator']
    print(f"  Total Generators: {len(gens)}")
    
    # Check carrier
    elec = builder.df_cap_full[builder.df_cap_full['carrier'] == 'electricity final use']
    print(f"  Total 'electricity final use': {len(elec)}")
    
    # Check area
    pl = builder.df_cap_full[builder.df_cap_full['area'] == 'PL']
    print(f"  Total PL components: {len(pl)}")
    
    # Check what areas have electricity final use
    elec_areas = elec['area'].unique()
    print(f"  Areas with 'electricity final use': {elec_areas}")


Matching rows: 0

No matches! Checking each filter separately...
  Total Generators: 80
  Total 'electricity final use': 5
  Total PL components: 148
  Areas with 'electricity final use': ['CZ' 'DE' 'LT' 'SE' 'SK']


### 🔍 Understanding Demand Components

The demand generator has:
- **sign = -1**: Indicates this is consumption, not generation
- **p_nom < 0**: Negative capacity represents demand
- **p_set**: Time-varying demand profile (from timeseries data)
- **bus**: Connected to "PL electricity in" bus

Now let's check the balance again:

In [17]:
# Check balance after adding demand
builder.inspect('balance')


SUPPLY/DEMAND BALANCE
Total supply capacity:    1014569.00 MW
  From Generators:         999999.00 MW
  From Links:               14570.00 MW (thermal plants)
Total demand capacity:      20969.38 MW
Balance ratio:                 48.38 (4838.3%)

⚠️  WARNING: Significant excess supply capacity

Generator Supply by Carrier
---------------------------
  hard coal supply                        :  999999.00 MW

Link Supply by Technology (Thermal Plants)
------------------------------------------
  hard coal power SC                      :    4050.00 MW
  hard coal power old                     :   10520.00 MW

Demand by Carrier
-----------------
  electricity HMV final use               :   12152.19 MW
  electricity LV final use                :    8817.19 MW


### ⚠️ IMBALANCE DETECTED!

You should see:
- **Supply**: ~X MW (from coal plants)
- **Demand**: ~Y MW (much larger!)
- **Balance ratio**: < 1.0 (probably around 0.3-0.5)
- **Warning**: Insufficient supply capacity!

This means we have **way more demand than supply**. The model is currently infeasible - optimization would fail!

Let's also run formal validation:

In [18]:
# Validate the network state
# This runs automated checks:
# - structure: Are buses/carriers/snapshots present?
# - balance: Is supply >= demand?

results = builder.validate_stage(
    stage_name='after_adding_demand',
    validation_types=['structure', 'balance']
)

# Show balance info
print("\nBalance info:")
for key, value in results['balance']['info'].items():
    print(f"  {key}: {value}")

2026-02-08 19:14:19 [INFO] 
Validation results for stage 'after_adding_demand':
2026-02-08 19:14:19 [INFO]   structure: ok
2026-02-08 19:14:19 [INFO]   balance: warning
2026-02-08 19:14:19 [WARNING]     - Excess supply: 4838.3% of demand capacity



Balance info:
  supply_capacity_MW: 1014569.0
  demand_capacity_MW: 20969.383561643837
  balance_ratio: 48.38334884845159


---

## Step 5: Try Optimization (Will Fail!)

Let's try to optimize this imbalanced model to see what happens.

In [19]:
# Attempt optimization
# This will:
# 1. Create optimization problem (minimize total system cost)
# 2. Add constraints (energy balance, capacity limits, etc.)
# 3. Call HiGHS solver
# 4. Return success/failure

print("\nAttempting optimization with insufficient capacity...")
print("(This should fail - we don't have enough generation!)\n")

success = builder.optimize()

if not success:
    print("\n❌ As expected, optimization failed!")
    print("   Reason: Insufficient generation capacity to meet demand")
    print("   We need to add more generation sources.")
else:
    print("\n⚠️ Unexpected: Optimization succeeded with insufficient capacity!")
    print("   This might mean demand is flexible or there are unaccounted sources.")

2026-02-08 19:14:19 [INFO] ============================================================
2026-02-08 19:14:19 [INFO] Running optimization...
2026-02-08 19:14:19 [INFO] ============================================================
2026-02-08 19:14:19 [INFO] Repeating time-series for each investment period and converting snapshots to a pandas.MultiIndex.
2026-02-08 19:14:19 [WARNING] The following buses have carriers which are not defined:
Index(['CZ battery large electricity', 'CZ biomass wood', 'CZ electricity in',
       'CZ electricity out', 'CZ hard coal', 'CZ hydro PSH electricity',
       'CZ hydrogen', 'CZ lignite', 'CZ natural gas',
       'DE battery large electricity', 'DE biomass wood', 'DE electricity in',
       'DE electricity out', 'DE hard coal', 'DE hydro PSH electricity',
       'DE hydrogen', 'DE lignite', 'DE natural gas',
       'LT battery large electricity', 'LT biomass wood', 'LT electricity in',
       'LT electricity out', 'LT hydro PSH electricity', 'LT hydrogen'


Attempting optimization with insufficient capacity...
(This should fail - we don't have enough generation!)



2026-02-08 19:14:19 [INFO]  Solve problem using Highs solver
2026-02-08 19:14:19 [INFO] Solver options:
 - threads: 8
 - solver: ipm
 - run_crossover: off
 - small_matrix_value: 1e-07
 - large_matrix_value: 1000000000000.0
 - ipm_optimality_tolerance: 1e-05
 - dual_feasibility_tolerance: 1e-05
 - primal_feasibility_tolerance: 0.0001
 - parallel: on
 - random_seed: 0
2026-02-08 19:14:19 [INFO]  Writing time: 0.05s
2026-02-08 19:14:19 [WARNING] Optimization potentially failed: 
Status: warning
Termination condition: infeasible
Solution: 0 primals, 0 duals
Objective: nan
Solver model: available
Solver message: infeasible

2026-02-08 19:14:19 [ERROR] Optimization failed! Status: warning: infeasible


Running HiGHS 1.9.0 (git hash: fa40bdf): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 3e+00]
  Cost   [3e+02, 3e+03]
  Bound  [0e+00, 0e+00]
  RHS    [9e+02, 1e+06]
Presolving model
Problem status detected on presolve: Infeasible
Model name          : linopy-problem-99twiqxj
Model status        : Infeasible
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
Writing the solution to /tmp/linopy-solve-q_9tkumn.sol

❌ As expected, optimization failed!
   Reason: Insufficient generation capacity to meet demand
   We need to add more generation sources.


### 🎓 Learning Point: Feasibility

**Infeasible models** cannot be solved because constraints cannot be satisfied simultaneously. Common causes:
1. **Insufficient capacity**: More demand than generation
2. **Disconnected network**: Some areas have no supply
3. **Conflicting constraints**: e.g., min generation > max capacity
4. **Temporal mismatch**: Even if total capacity is enough, timing might not work (e.g., all solar, no storage, high night demand)

The balance check helps catch issue #1 before optimization.

---

## Step 6: Add Renewable Generation

Let's add wind and solar to increase supply capacity. **These are Generators**, not Links!

In [20]:
# Add renewable generation
#
# Component type: Generator (NOT Link!)
# Why: Renewables directly inject electricity into the grid
#      No fuel conversion - wind/sun → electricity
#      They have variable output (p_max_pu time series)
#
# Technologies:
# - 'wind onshore': Land-based wind turbines
# - 'solar PV ground': Ground-mounted solar panels
#
# Key attributes:
# - p_nom: Installed capacity (MW)
# - p_max_pu: Availability factor (0-1, time-varying)
#             Based on weather data (wind speed, solar irradiance)

summary = builder.add_components(
    'Generator',  # Component type: Generator (direct injection)
    {
        'technology': ['wind onshore', 'solar PV ground'],
        'area': ['PL']  # Only Poland - exclude foreign countries
    }
)

print(f"\n✓ Added {summary['added']} renewable generation components")
print(f"  Total Generators in network: {summary['generators']}")

2026-02-08 19:14:19 [INFO] Defining time-dependent attributes...
2026-02-08 19:14:19 [INFO] Adding 4 components to network...
2026-02-08 19:14:20 [INFO] Added 4 components. Network now has: 7 generators, 2 links, 0 lines, 0 stores



✓ Added 4 renewable generation components
  Total Generators in network: 7


### Now Add Gas Plants

Gas plants provide flexible, dispatchable generation. Like coal, **they are Links**.

In [21]:
# Add gas fuel supply + power plants
#
# CRITICAL: Thermal Links need FUEL to operate!
# Step 1: Add unlimited fuel supply (Generator)
# Step 2: Add power plants (Link) that convert fuel → electricity

# Step 1: Add gas fuel supply
print("Step 1: Adding gas fuel supply...")
summary = builder.add_components(
    'Generator',
    {
        'technology': ['natural gas supply'],  # Provides gas to 'PL natural gas' bus
        'area': ['PL']
    }
)
print(f"  ✓ Added {summary['added']} fuel supply components")

# Step 2: Add gas power plants
print("\nStep 2: Adding gas power plants...")
summary = builder.add_components(
    'Link',  # Component type: Link (fuel conversion)
    {
        'technology': ['natural gas power CCGT'],
        'area': ['PL']  # Only Poland - exclude foreign countries
    }
)

print(f"  ✓ Added {summary['added']} gas power plant components")
print(f"\nTotal network components:")
print(f"  Generators: {summary['generators']}")
print(f"  Links: {summary['links']}")

2026-02-08 19:14:20 [INFO] Defining time-dependent attributes...
2026-02-08 19:14:20 [INFO] Adding 1 components to network...


Step 1: Adding gas fuel supply...


2026-02-08 19:14:20 [INFO] Added 1 components. Network now has: 8 generators, 2 links, 0 lines, 0 stores
2026-02-08 19:14:20 [INFO] Defining time-dependent attributes...
2026-02-08 19:14:20 [INFO] Adding 1 components to network...


  ✓ Added 1 fuel supply components

Step 2: Adding gas power plants...


2026-02-08 19:14:21 [INFO] Added 1 components. Network now has: 8 generators, 3 links, 0 lines, 0 stores


  ✓ Added 1 gas power plant components

Total network components:
  Generators: 8
  Links: 3


### Check Balance Again

In [22]:
# Check balance after adding renewables and gas
builder.inspect('balance')


SUPPLY/DEMAND BALANCE
Total supply capacity:    2039628.00 MW
  From Generators:        2022938.00 MW
  From Links:               16690.00 MW (thermal plants)
Total demand capacity:      20969.38 MW
Balance ratio:                 97.27 (9726.7%)

⚠️  WARNING: Significant excess supply capacity

Generator Supply by Carrier
---------------------------
  hard coal supply                        :  999999.00 MW
  natural gas supply                      :  999999.00 MW
  solar PV ground                         :   12290.00 MW
  wind onshore                            :   10650.00 MW

Link Supply by Technology (Thermal Plants)
------------------------------------------
  hard coal power SC                      :    4050.00 MW
  hard coal power old                     :   10520.00 MW
  natural gas power CCGT                  :    2120.00 MW

Demand by Carrier
-----------------
  electricity HMV final use               :   12152.19 MW
  electricity LV final use                :    8817.19 MW


### ✅ BALANCED!

You should now see:
- **Supply from Generators**: ~X MW (wind + solar)
- **Supply from Links**: ~Y MW (coal + gas)
- **Total Supply**: X + Y MW
- **Demand**: ~Z MW
- **Balance ratio**: ~1.2-1.5 (good! slightly oversupplied for reliability)
- **Status**: ✓ Supply and demand are reasonably balanced

Let's also look at the detailed breakdown:

In [23]:
# Show detailed breakdown of all components
builder.inspect('detailed')


DETAILED BREAKDOWN

Generators by Technology
------------------------
  electricity HMV final use               :   1 units,   12152.19 MW
  electricity LV final use                :   1 units,    8817.19 MW
  hard coal supply                        :   1 units,  999999.00 MW
  natural gas supply                      :   1 units,  999999.00 MW
  solar PV ground                         :   2 units,   12290.00 MW
  wind onshore                            :   1 units,    4390.00 MW
  wind onshore old                        :   1 units,    6260.00 MW

Links by Technology
-------------------
  hard coal power SC                      :   1 units,    4050.00 MW
  hard coal power old                     :   1 units,   10520.00 MW
  natural gas power CCGT                  :   1 units,    2120.00 MW


### 📊 Reading the Detailed Report

- **Generators by Technology**: Shows renewable capacity
- **Links by Technology**: Shows thermal plant capacity

Notice how the report now correctly separates Generators and Links! This is one of the critical fixes that was applied.

---

## Step 7: Optimize the Balanced Model

Now that we have sufficient capacity, let's optimize!

In [24]:
# Run optimization
# This will:
# 1. Formulate linear programming problem:
#    Objective: Minimize (investment_cost + variable_cost + fixed_cost)
#    Subject to:
#    - Energy balance at each bus and timestep
#    - Capacity limits (0 <= p <= p_nom)
#    - Efficiency constraints for links
#    - Storage state-of-charge dynamics
#    - Any global constraints (CO2, etc.)
#
# 2. Call HiGHS solver (open-source, fast)
#
# 3. Extract results:
#    - Optimal dispatch: generators_t.p, links_t.p
#    - Optimal investment: if extendable
#    - Prices: buses_t.marginal_price
#    - Objective value: total system cost

print("\nOptimizing balanced model...")
print("This may take 30-60 seconds for mini timeseries.\n")

success = builder.optimize()

if success:
    print("\n✅ Optimization succeeded!")
    print("   The model found a feasible, cost-optimal solution.")
else:
    print("\n❌ Optimization failed!")
    print("   Check logs above for details.")

2026-02-08 19:14:21 [INFO] ============================================================
2026-02-08 19:14:21 [INFO] Running optimization...
2026-02-08 19:14:21 [INFO] ============================================================
2026-02-08 19:14:21 [WARNING] The following buses have carriers which are not defined:
Index(['CZ battery large electricity', 'CZ biomass wood', 'CZ electricity in',
       'CZ electricity out', 'CZ hard coal', 'CZ hydro PSH electricity',
       'CZ hydrogen', 'CZ lignite', 'CZ natural gas',
       'DE battery large electricity', 'DE biomass wood', 'DE electricity in',
       'DE electricity out', 'DE hard coal', 'DE hydro PSH electricity',
       'DE hydrogen', 'DE lignite', 'DE natural gas',
       'LT battery large electricity', 'LT biomass wood', 'LT electricity in',
       'LT electricity out', 'LT hydro PSH electricity', 'LT hydrogen',
       'LT natural gas', 'PL BEV electricity', 'PL ICE vehicle fuel',
       'PL battery large electricity HMV',
       'PL


Optimizing balanced model...
This may take 30-60 seconds for mini timeseries.



2026-02-08 19:14:21 [INFO]  Solve problem using Highs solver
2026-02-08 19:14:21 [INFO] Solver options:
 - threads: 8
 - solver: ipm
 - run_crossover: off
 - small_matrix_value: 1e-07
 - large_matrix_value: 1000000000000.0
 - ipm_optimality_tolerance: 1e-05
 - dual_feasibility_tolerance: 1e-05
 - primal_feasibility_tolerance: 0.0001
 - parallel: on
 - random_seed: 0
2026-02-08 19:14:22 [INFO]  Writing time: 0.09s
2026-02-08 19:14:22 [WARNING] Optimization potentially failed: 
Status: warning
Termination condition: infeasible
Solution: 0 primals, 0 duals
Objective: nan
Solver model: available
Solver message: infeasible

2026-02-08 19:14:22 [ERROR] Optimization failed! Status: warning: infeasible


Running HiGHS 1.9.0 (git hash: fa40bdf): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 3e+00]
  Cost   [2e+02, 3e+03]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 1e+06]
Presolving model
Problem status detected on presolve: Infeasible
Model name          : linopy-problem-xz4585ai
Model status        : Infeasible
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.01
Writing the solution to /tmp/linopy-solve-q856seh_.sol

❌ Optimization failed!
   Check logs above for details.


### 🎓 Understanding the Solver Output

The HiGHS solver log shows:
- **Variables**: Number of decision variables (generator dispatch, storage levels, etc.)
- **Constraints**: Number of equations to satisfy
- **Iterations**: How many steps the solver took
- **Objective**: Total system cost (EUR)
- **Status**: "ok" = optimal solution found

Let's inspect the results:

In [25]:
# Show optimization results
builder.inspect('optimization')


OPTIMIZATION RESULTS
Solver status: warning: infeasible

❌ Optimization failed or is infeasible


### 📊 Understanding the Results

- **Solver Status**: Should be "ok"
- **Objective Value**: Total system cost (investment + operation) in EUR
- **Total Energy Generated**: Sum across all technologies and timesteps (MWh)
- **Generation by Technology**: Shows which technologies were used and how much

**Key Insights**:
1. Renewables will be used at maximum availability (they have zero marginal cost)
2. Coal/gas will fill the gaps when renewables are low
3. Gas might be preferred over coal due to higher efficiency (less fuel cost per MWh)

Let's look at the complete picture:

In [26]:
# Show everything: summary, detailed breakdown, balance, and optimization
builder.inspect('all')


NETWORK SUMMARY
Network components:
  Areas:      6
  Buses:      92
  Carriers:   106
  Snapshots:  672
  Generators: 8
  Links:      3
  Lines:      0
  Stores:     0
  Global constraints: 0

DETAILED BREAKDOWN

Generators by Technology
------------------------
  electricity HMV final use               :   1 units,   12152.19 MW
  electricity LV final use                :   1 units,    8817.19 MW
  hard coal supply                        :   1 units,  999999.00 MW
  natural gas supply                      :   1 units,  999999.00 MW
  solar PV ground                         :   2 units,   12290.00 MW
  wind onshore                            :   1 units,    4390.00 MW
  wind onshore old                        :   1 units,    6260.00 MW

Links by Technology
-------------------
  hard coal power SC                      :   1 units,    4050.00 MW
  hard coal power old                     :   1 units,   10520.00 MW
  natural gas power CCGT                  :   1 units,    2120.00 MW

SUP

---

## Step 8: Save Your Work

You can save the current network state as a checkpoint for later use.

In [ ]:
# Save checkpoint
# This saves:
# - Network components (all CSV files)
# - Capacity data
# - Time-dependent attributes
# - Stage history
#
# Location: ./checkpoints/<checkpoint_name>/

checkpoint_name = 'tutorial_basic_balanced_model'

builder.save_checkpoint(checkpoint_name)

print(f"\n✓ Checkpoint saved: {checkpoint_name}")
print("  You can load this later with: builder.load_checkpoint('{checkpoint_name}')")

---

## 🎯 Summary: What We Learned

### 1. Component Types Matter!
- **Generators**: Renewables, nuclear, demand (negative)
- **Links**: Thermal plants, heat pumps, conversion technologies
- **Lines**: Transmission infrastructure
- **Stores**: Energy storage

### 2. Technology Names Use Spaces
- ✓ `'hard coal power old'`, `'wind onshore'`
- ✗ `'hard_coal_power_old'`, `'wind_onshore'`

### 3. Balance Before Optimizing
- Check `builder.inspect('balance')` before optimization
- Need supply ≥ demand (ideally 1.2-1.5x for reliability)
- Balance includes BOTH Generators AND Links

### 4. Incremental Building Helps Understanding
- Start simple: coal + demand (infeasible)
- Add renewables + gas (balanced)
- Optimize and analyze results
- See cause-and-effect of each addition

### 5. Optimization Needs Feasibility
- Sufficient capacity
- Network connectivity
- Temporal matching (renewables + storage or dispatchable backup)

---

## 🚀 Next Steps

Now that you understand the basics, try these exercises:

### Exercise 1: Add Storage
```python
# Add battery storage
builder.add_components('Store', {
    'technology': ['battery large storage'],
    'area': ['PL']
})

# Re-optimize and compare
builder.optimize()
builder.inspect('optimization')
```

### Exercise 2: Add Heat Sector
```python
# Add heat demand
builder.add_components('Generator', {
    'carrier': ['space heating final use'],
    'area': ['PL']
})

# Add heat pumps (Links!)
builder.add_components('Link', {
    'technology': ['heat pump large AW'],
    'area': ['PL']
})
```

### Exercise 3: Compare Scenarios
```python
# Build another model without renewables
builder2 = IncrementalBuilder(year=2025, timeseries='mini')
builder2.build_base_model()
# Add only coal and gas...
# Compare costs and emissions
```

### Exercise 4: Explore Technologies
```python
# See all available technologies
from pypsa_pl.build_network import process_capacity_data
df = process_capacity_data(builder.inputs, builder.params)

# Group by component type
for comp in ['Generator', 'Link', 'Store', 'Line']:
    print(f"\n{comp}:")
    techs = df[df.component == comp].technology.unique()
    for tech in sorted(techs)[:10]:  # Show first 10
        print(f"  - {tech}")
```

---

## 📚 Additional Resources

### Documentation
- **[COMPONENT_REFERENCE.md](COMPONENT_REFERENCE.md)**: Complete mapping of all technologies
- **[README.md](README.md)**: API reference and examples
- **[GETTING_STARTED.md](GETTING_STARTED.md)**: Quick start guide
- **[PLAN.md](PLAN.md)**: Detailed design documentation

### PyPSA Documentation
- Official docs: https://pypsa.readthedocs.io/
- Examples: https://pypsa.readthedocs.io/en/latest/examples-basic.html

### Tips for Learning
1. **Start simple**: Use `timeseries='mini'` and `copperplate=True`
2. **Check balance often**: Before and after adding components
3. **Use validation**: `builder.validate_stage()` catches issues early
4. **Save checkpoints**: Don't lose working models
5. **Read the logs**: They tell you exactly what's happening

---

## 🎉 Congratulations!

You've successfully built a PyPSA model incrementally from scratch. You now understand:
- ✅ Component types and when to use each
- ✅ How to build models step-by-step
- ✅ How to validate and inspect at each stage
- ✅ What makes a model feasible or infeasible
- ✅ How to interpret optimization results

Keep experimenting and learning! 🚀